In [94]:
import torch
import numpy as np
import torch.nn as nn
from sklearn.metrics import f1_score, recall_score, precision_score
from torch.utils.data import DataLoader, TensorDataset
import pandas as pd
import sklearn
import json
from scipy.io import mmread, mmwrite, mminfo
import os
import pickle
import torch.nn.functional as F

In [2]:
corp_data = torch.load('ChinaCorp_1x_2023-06-06.pt')
all_data = pd.read_csv('../train_test_x1.csv', encoding='gb18030')
listed_corp = json.load(open('../meta-path/listed_firm.json', 'r'))
test = all_data[all_data['pred_year']>=2020]

# seed = 2027 时效果最好
path_attn = np.load('results/base-han_5e-05_0.0_0.001_[2, 2]_256_30_[0]_512_64_None_2023-06-07/path_attn_2027.npy')
y_pred = np.loadtxt('results/base-han_5e-05_0.0_0.001_[2, 2]_256_30_[0]_512_64_None_2023-06-07/y_pred_2027.txt')

path_attn.shape, y_pred.shape, listed_corp.index(597), test[['Stkcd','pred_year']].values.tolist().index([597, 2021])

with open('results/base-han_5e-05_0.0_0.001_[2, 2]_256_30_[0]_512_64_None_2023-06-07/gat_attn_2027.pt', 'rb') as f:
    gat_attn = pickle.load(f)

In [77]:
fraud_firm_year = all_data[all_data['label']==1][['Stkcd','pred_year']].values.tolist()

In [53]:
gat_attn[0][0][0, 509].item()

0.18706557154655457

In [263]:
corp_data['graphs'][2003]['adj'][0].all_edges()

(tensor([  44,   52,   36,   36,   53, 1441, 1439, 1441, 1442, 1441, 3027,   52,
          288,   96,   62, 2352, 2851, 2541, 2926,  274,  471,  411, 2622, 2315,
         2592, 2301, 2486, 2614, 2315, 2865, 2433, 2404, 2860, 2853, 2955,  358,
         2373, 2315, 2746, 3295, 2782, 2486, 2886, 2893,  238, 2365,  314, 2486,
         2886, 3295, 2442,  323, 2931, 2966, 2462, 2315, 3055,  687, 2926, 2352,
         2900,  421, 2604, 2902, 2315, 2639, 2589, 2945, 1653, 2376, 2810, 2949,
         2443, 2443, 2579, 2357, 3030, 3078, 2530, 3071, 2966, 2842, 2787, 2972,
         2431, 3054, 2480, 2556, 2540, 2886, 2984, 2960, 2504, 3023, 2929, 2579,
         2594, 2922, 2817, 3822, 2781, 3013, 2719, 3295, 2861, 2866, 2847, 2830,
         2809, 2846, 2831, 2821, 3020, 2840, 2818, 2819, 3018, 2969, 2869, 2880,
         2848, 3484, 2880, 3027, 2930, 2866, 3290, 2898, 2960, 3028, 3295, 4100,
         4109, 3034,    5,    5,    9,   11,   12,   13,   15,   15,   18,   38,
           39,   44,   51,  

000513 预测概率y_pred是

In [149]:
class Analyzer(object):
    def __init__(self):
        self.corp_data = torch.load('ChinaCorp_1x_2023-06-07.pt')
        self.listed_corp = json.load(open('../meta-path/listed_firm.json', 'r'))
        self.all_data = pd.read_csv('../train_test_x1.csv', encoding='gb18030')
        # all_data = all_data.set_index(['Stkcd', 'pred_year'])
        # 2020 年以后的是测试集
        self.test = self.all_data[self.all_data['pred_year']>=2020]
        self.output_dir = 'results/base-han_5e-05_0.0_0.001_[2, 2]_256_30_[0]_512_64_None_2023-06-07'
        # test = corp_data['Stkcd_year']
        self.path_list = [
            # 1-hop
            'guanlian', 'gudong',
            # 2-hop
            'gud(T)_gud', 'gud(T)_trans', 'gud(T)_mana', 'trans_mana', 'gud_trans', 'mana(T)_mana', 'gud_gud(T)',
            # 3-hop
            'gu(T)_qin_gu', 'gu(T)_qin_trans', 'gu(T)_qin_mana', 'trans_qin_trans','trans_qin_mana', 'mana(T)_qin_mana',
            'mana(T)_trans_trans', 'trans_mana(T)_trans',
            'gud_trans_mana', 'gud(T)_trans_mana',
            'trans_gud(T)_mana', 'gud(T)_mana_trans',
            'trans_mana_gud(T)', 'mana(T)_mana_trans',
            'mana(T)_mana_gud', 'mana(T)_mana_gud(T)'
        ]

    def analyze_meta_path_attn(self, stk_id, year):
        
        self.attn_mat = np.load(f'{self.output_dir}/path_attn_2027.npy')
        self.y_pred = np.loadtxt(f'{self.output_dir}/y_pred_2027.txt')
        
        print(f'待分析的公司代码 {stk_id}，年份 {year}')
        # 先获取 firm-year 在 meta-path矩阵中的下标，以及在训练样本中的下标
        self.index_in_mat = self.listed_corp.index(stk_id)
        self.index_in_test = self.test[['Stkcd','pred_year']].values.tolist().index([stk_id, year])
        # 输入模型对于此样本的预测概率
        print(self.y_pred[self.index_in_test])
        
        self.path_end_corp = {} # 目标公司是头节点
        self.path_begin_corp = {} # 目标公司是尾节点
        
        for path in self.path_list:
            coo = mmread(f'../meta-path/mat_by_year/{year}/metapath/{path}.mtx')
            # print(f'------------------------{path}------------------------')

            row_data = coo.getrow(self.index_in_mat).toarray().flatten()
            end_values = row_data[row_data.nonzero()]
            end_indices = np.nonzero(row_data)[0]

            if len(end_values) > 0: # 有元路径的才保留
                self.path_end_corp[path] = [self.listed_corp[i] for i in end_indices]
                
            # 将元路径反过来，将当前公司当成尾节点
            col_data = coo.getcol(self.index_in_mat).toarray().flatten()
            
            start_values = row_data[col_data.nonzero()]
            start_indices = np.nonzero(col_data)[0]

            if len(start_values) > 0: # 有元路径的才保留
                self.path_begin_corp[path] = [self.listed_corp[i] for i in start_indices]

            
        print(f'有实例的 meta-path 路径有：{list(self.path_end_corp.keys())}')
        print(f'有实例的 反meta-path 路径有：{list(self.path_begin_corp.keys())}')
        self.path_attn = {}

        for path in list(self.path_end_corp.keys())+list(self.path_begin_corp.keys()):
            path_id = self.path_list.index(path)
            value = self.attn_mat[year-2003][self.index_in_mat][path_id]
            self.path_attn[path] = value
            
        print(self.path_attn)

        return self.path_attn, self.path_end_corp, self.path_begin_corp

    def analyze_gat_attn(self, stk_code, year, path_name):
        ngh_list = self.path_end_corp[path_name]

        with open(f'{self.output_dir}/gat_attn_2027.pt', 'rb') as f:
            self.gat_attn = pickle.load(f)
        
        if year < 2020:
            print(f'year {year} error')
            return
        
        # coo matrix: (4111, 4111)
        coo_gat_attn = self.gat_attn[year - 2020][self.path_list.index(path_name)]
        
        stk_id = self.listed_corp.index(stk_code)
        # ngh_ids = [self.listed_corp.index(i) for i in ngh_list]

        path_weight = {}    
        # 将 当前提出的所有 路径实例 的gat-attn做softmax归一化
        unnormed_attn = [coo_gat_attn[stk_id, self.listed_corp.index(i)].item() for i in ngh_list]
        normed_attn = F.softmax(torch.tensor(unnormed_attn), dim=-1).tolist()
        
        for i in range(len(ngh_list)):
            ngh = ngh_list[i]
            attn_value = normed_attn[i]
            path_weight[(stk_code, ngh)] = attn_value

        return path_weight

In [145]:
# 700, 2020 可以
# 2074, 2020 比较多，但没有 qinshu
# 2076, 2021 有 qinshu
# 2089, 2020 有 qinshu
# 2175, 2020 有 qinshu，并且 end entity是别的公司
ana = Analyzer()
path_attn, path_end_corp, path_begin_corp = ana.analyze_meta_path_attn(700, 2020)
ana.index_in_mat

待分析的公司代码 700，年份 2020
[-0.25292724 -0.08369462]
有实例的 meta-path 路径有：['gudong', 'gud(T)_gud', 'gud(T)_trans', 'mana(T)_mana', 'gu(T)_qin_gu', 'gu(T)_qin_mana', 'mana(T)_mana_gud', 'mana(T)_mana_gud(T)']
有实例的 反meta-path 路径有：['gud(T)_gud', 'gud(T)_trans', 'mana(T)_mana', 'gu(T)_qin_gu', 'mana(T)_mana_gud(T)']
{'gudong': 0.02562753, 'gud(T)_gud': 0.0094306925, 'gud(T)_trans': 0.024001991, 'mana(T)_mana': 0.07095009, 'gu(T)_qin_gu': 0.07003452, 'gu(T)_qin_mana': 0.64323056, 'mana(T)_mana_gud': 0.15470119, 'mana(T)_mana_gud(T)': 0.0020234548}


256

In [121]:
ana.analyze_gat_attn(700, 2020, 'gud(T)_gud')

{(700, 2): 0.0008480479591526091,
 (700, 6): 0.0008478806121274829,
 (700, 8): 0.0008477619849145412,
 (700, 9): 0.0008475923677906394,
 (700, 11): 0.0008473688503727317,
 (700, 12): 0.0008479242096655071,
 (700, 16): 0.0008481599506922066,
 (700, 19): 0.0008482862031087279,
 (700, 21): 0.0008476543589495122,
 (700, 25): 0.0008476537768729031,
 (700, 27): 0.0008482946432195604,
 (700, 28): 0.0008475907961837947,
 (700, 29): 0.0008482730481773615,
 (700, 31): 0.0008475264185108244,
 (700, 32): 0.0008477037190459669,
 (700, 34): 0.0008476712973788381,
 (700, 36): 0.0008473738562315702,
 (700, 49): 0.0008472289191558957,
 (700, 50): 0.0008481903350912035,
 (700, 58): 0.0008479540701955557,
 (700, 59): 0.0008482292178086936,
 (700, 60): 0.0008478785748593509,
 (700, 61): 0.0008480874239467084,
 (700, 62): 0.0008475958020426333,
 (700, 63): 0.0008477861993014812,
 (700, 65): 0.0008478640811517835,
 (700, 66): 0.0008474139031022787,
 (700, 69): 0.000848163734190166,
 (700, 70): 0.00084738340

In [70]:
path_end_corp['mana(T)_mana_gud']

[601199]

In [119]:
# 分析多个ngh里面有欺诈的公司的gat权重
gudTgud = ana.analyze_gat_attn(700, 2020, 'gud(T)_gud')
for target, firm in gudTgud:
    if [firm, 2020] in fraud_firm_year:
        print(firm, gudTgud[target, firm])

100 0.0008471690816804767
408 0.001048340112902224
676 0.0008481447584927082
910 0.000847533461637795
2065 0.0008474995847791433
2074 0.000847263028845191
2217 0.000847843533847481
2285 0.0008478113450109959
2303 0.0008470942848362029
2366 0.0008479934767819941
2387 0.0008478173986077309
2618 0.0008477297960780561
2625 0.00084810791304335
300257 0.0008478715317323804
300300 0.0008482387638650835
600070 0.0008477202500216663
600310 0.0008479418465867639
600586 0.0008476010407321155
600711 0.0008479589596390724
600748 0.000847749353852123
603766 0.0008478573290631175


In [150]:
ana = Analyzer()
path_attn, path_end_corp, path_begin_corp = ana.analyze_meta_path_attn(2175, 2020)
ana.index_in_mat

待分析的公司代码 2175，年份 2020
[-0.42527482  0.34802836]
有实例的 meta-path 路径有：['gud(T)_gud', 'gud(T)_trans', 'mana(T)_mana', 'gu(T)_qin_gu', 'mana(T)_mana_gud', 'mana(T)_mana_gud(T)']
有实例的 反meta-path 路径有：['gud(T)_gud', 'gud(T)_trans', 'mana(T)_mana', 'gu(T)_qin_gu']
{'gud(T)_gud': 0.2302623, 'gud(T)_trans': 0.38471842, 'mana(T)_mana': 0.12418382, 'gu(T)_qin_gu': 0.08927204, 'mana(T)_mana_gud': 0.030309666, 'mana(T)_mana_gud(T)': 0.1412538}


657

In [153]:
ana.analyze_gat_attn(2175, 2020, 'mana(T)_mana')

{(2175, 669): 0.19910548627376556,
 (2175, 2175): 0.2184418886899948,
 (2175, 300108): 0.1950584053993225,
 (2175, 300302): 0.1918083280324936,
 (2175, 603569): 0.19558589160442352}

In [142]:
# 分析多个ngh里面有欺诈的公司的gat权重
gudTgud = ana.analyze_gat_attn(2175, 2020, 'gud(T)_gud')
for target, firm in gudTgud:
    if [firm, 2020] in fraud_firm_year:
        print(firm, gudTgud[target, firm])

2089 0.054804518818855286


In [134]:
test[:50]

,Stkcd,pred_year,time_series,label,industry,asset
80,100,2021,"[2021, 2020, 2019, 2018, 2017, 2016]",1,工业,3.087331e+11
81,601992,2021,"[2021, 2020, 2019, 2018, 2017, 2016]",0,工业,2.863568e+11
82,100,2020,"[2020, 2019, 2018, 2017, 2016]",1,工业,2.579083e+11
83,651,2020,"[2020, 2019, 2018, 2017, 2016, 2015]",0,工业,2.792179e+11
106,408,2020,"[2020, 2019, 2018, 2017, 2016, 2015]",1,工业,8.677639e+09
107,2946,2020,"[2020, 2019]",0,工业,8.578820e+09
144,523,2020,"[2020, 2019, 2018, 2017, 2016, 2015]",1,工业,3.399414e+09
145,600101,2020,"[2020, 2019, 2018, 2017, 2016, 2015]",0,工业,3.401386e+09
242,638,2021,"[2021, 2020, 2019, 2018, 2017, 2016]",1,公用事业,5.604279e+08
243,300668,2021,"[2021, 2020, 2019, 2018]",0,公用事业,6.057752e+08


In [182]:
with open('../meta-path/listed_firm.json','r') as f:
    listed_corp = json.load(f)
with open('../meta-path/unlisted_firm.json', 'r') as f:
    unlisted_corp = json.load(f)
with open('../meta-path/people.json','r') as f:
    people = json.load(f)

n_listed_corp = len(listed_corp)
n_unlisted_corp = len(unlisted_corp)
n_people = len(people)
n_all = n_listed_corp + n_unlisted_corp + n_people

all_body = listed_corp + unlisted_corp + people
print(len(all_body))

541521


In [122]:
path_attn, path_end_corp, path_begin_corp = ana.analyze_meta_path_attn(600243, 2021)

待分析的公司代码 600243，年份 2021
[-0.46847606 -0.06158544]
有实例的 meta-path 路径有：['guanlian', 'gud(T)_gud', 'gud(T)_trans', 'gud(T)_mana', 'trans_mana', 'gud_trans', 'mana(T)_mana', 'mana(T)_trans_trans', 'trans_mana(T)_trans', 'trans_gud(T)_mana', 'gud(T)_mana_trans', 'mana(T)_mana_trans', 'mana(T)_mana_gud', 'mana(T)_mana_gud(T)']
有实例的 反meta-path 路径有：['guanlian', 'gud(T)_gud', 'gud(T)_trans', 'gud(T)_mana', 'trans_mana', 'gud_trans', 'mana(T)_mana', 'mana(T)_trans_trans', 'trans_mana(T)_trans', 'trans_gud(T)_mana', 'gud(T)_mana_trans', 'mana(T)_mana_trans']
{'guanlian': 0.13351487, 'gud(T)_gud': 0.008648416, 'gud(T)_trans': 0.037204694, 'gud(T)_mana': 0.052235123, 'trans_mana': 0.09945541, 'gud_trans': 0.017784778, 'mana(T)_mana': 0.030930491, 'mana(T)_trans_trans': 0.07497749, 'trans_mana(T)_trans': 0.08083734, 'trans_gud(T)_mana': 0.066413194, 'gud(T)_mana_trans': 0.016165983, 'mana(T)_mana_trans': 0.028643778, 'mana(T)_mana_gud': 0.28761894, 'mana(T)_mana_gud(T)': 0.06556954}


In [125]:
path_end_corp

{'guanlian': [600243],
 'gud(T)_gud': [7,
  668,
  695,
  861,
  2336,
  2674,
  2781,
  300521,
  600117,
  600265,
  600421,
  600605,
  600992,
  603958],
 'gud(T)_trans': [600243],
 'gud(T)_mana': [600243, 600458],
 'trans_mana': [600243],
 'gud_trans': [600243],
 'mana(T)_mana': [700, 2903, 300449, 600243, 601168, 603616],
 'mana(T)_trans_trans': [600243],
 'trans_mana(T)_trans': [600243],
 'trans_gud(T)_mana': [600243, 600458],
 'gud(T)_mana_trans': [600243, 601766, 688187],
 'mana(T)_mana_trans': [600117, 600243],
 'mana(T)_mana_gud': [601199],
 'mana(T)_mana_gud(T)': [601288, 601398]}

In [123]:
ana.analyze_gat_attn(600243,2021,'guanlian')

{(600243, 600243): 1.0}